## Borz0 1.2.0

## Initializing :

In [18]:
#Importing the os, Spark in Python 3.12.7 and Checking the Version :
import os
from IPython.display import clear_output
import pandas as Pada
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
print("Pandas version :", Pada.__version__)
print("PySpark version :", pyspark.__version__)

Pandas version : 2.2.3
PySpark version : 3.5.4


##  Starting Borz0 1.2.0 :

In [19]:
#Build the ETL PipeLine
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11"

spark = SparkSession.builder \
    .appName("Simple Test") \
    .master("local[*]") \
    .getOrCreate()

print("Spark session created successfully!")

Spark session created successfully!


## Load Data :

In [20]:
#Ask User for the Data they want to use, and then load it after checking it exists:
while True:
    data_loaded = input("Please enter the Data you want to Load : ")
    if os.path.isfile(data_loaded):
        print(f"{data_loaded} exists in the current directory and will be loaded")
        break
    else:
        print(f"Fatal Error, {data_loaded} does not exist in the directory")

BankData.csv exists in the current directory and will be loaded


In [21]:
#Visualizing the Data
file_path = data_loaded
howMany_shown = int(input("Please enter how many rows you want to display"))
df_spark = spark.read.csv(file_path, header=True, inferSchema=True)
df_spark.show(howMany_shown)

+-----+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+
| name|lastname|               email|      phone|             address|balance|credit_nb|     MS| CS|                uuid|
+-----+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+
|Bruce|   Wayne|notbatman@gotham.com|        911|1007 Mountain Dri...|  9.0E9|     2581| single|850|d410999a-0a62-400...|
| Jack|Torrence|whatsanemail@what...|555-232-531|  Overlook Hotel, CO| 6000.0|        2|married|702|e1626813-e80c-4d8...|
+-----+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+
only showing top 2 rows



## Identifying the Parameters

In [22]:
df_spark.printSchema()

root
 |-- name: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- credit_nb: integer (nullable = true)
 |-- MS: string (nullable = true)
 |-- CS: integer (nullable = true)
 |-- uuid: string (nullable = true)



## Show With Parameter

In [23]:
#Show All Data Related to One Parameter

# Load the CSV into PySpark DataFrame
df = spark.read.csv(data_loaded, header=True, inferSchema=True)

#Asking the User about the Precise Data he wants to search within a certain Paramter
f_Param = input("Please enter The Parameter you want to sear in :")

#Initializing a GLOBAL Variable to treat the loaded Data like a SQL DataBase
df.createOrReplaceTempView("GLOBAL")

#Trying to display the Data Via a SQL Query
try:
    query = f"SELECT `{f_Param}` FROM GLOBAL"
    spark.sql(query).show()
except Exception as e: #saves the exception parameter in e
    print(f"Error executing query: {e}")





+--------------------+
|               email|
+--------------------+
|notbatman@gotham.com|
|whatsanemail@what...|
|  Fallout4@gmail.com|
|alice.smith@examp...|
|bob.brown@example...|
|charlie.j@example...|
| david.w@example.com|
|ella.white@exampl...|
|fiona.black@examp...|
|george.green@exam...|
|hannah.blue@examp...|
|isaac.red@example...|
|jackie.gold@examp...|
|kathy.silver@exam...|
|liam.gray@example...|
|mona.diamond@exam...|
|nina.copper@examp...|
|oscar.platinum@ex...|
|paula.steel@examp...|
|quincy.lead@examp...|
+--------------------+
only showing top 20 rows



## Searching via a parameter, a certain Data || Find Duplicates

In [25]:
#Search Algorythm, looking for a certain data like "SELECT * FROM clients WHERE name = Papatate"

# Load the CSV into PySpark DataFrame
df = spark.read.csv(data_loaded, header=True, inferSchema=True)

#Asking the User about the Precise Data he wants to search within a certain Paramter
f_Param = input("Please enter The Parameter you want to sear in :")
f_data = input(f"Please Enter the data you want to search with the {f_Param} parameter")

#Initializing a GLOBAL Variable to treat the loaded Data like a SQL DataBase
df.createOrReplaceTempView("GLOBAL")

#Try to Display the precise Data We're Looking for, as you can see, it also works to find Duplicates
try:
    query = f"SELECT * FROM GLOBAL WHERE `{f_Param}` = '{f_data}'"
    spark.sql(query).show()
except Exception as e: #saves the exception parameter in e
    print(f"Error executing query: {e}")

+------+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+
|  name|lastname|               email|      phone|             address|balance|credit_nb|     MS| CS|                uuid|
+------+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+
| Bruce|   Wayne|notbatman@gotham.com|        911|1007 Mountain Dri...|  9.0E9|     2581| single|850|d410999a-0a62-400...|
|Nathan|  Player|  Fallout4@gmail.com|555-233-431|2, Sanctuary Hill...|    0.0|        0|widowed|850|6998d527-b849-484...|
+------+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+



## Ordering Data with Parameter :

In [26]:
#Order Data With a parametter

#First, check if the parameter exists in the current Database
while True:
    parameter = input("Please enter the Parameter to use for Sorting the Data")
    clear_output(wait=True)
    if parameter in df_spark.columns:
        break
    else:
        print("Error, this parameter doesnt exist in the current csv file")

#Asking the User how many rows they want to display
howMany_shown = int(input("Please enter how many rows you want to display"))

#Choosing if the user wants to use desc or asc to determine the order of the Data used as a point of reference
while True:
    param2 = input("Do you want to start with the asc (type = asc) or the biggest (type = desc)")
    clear_output(wait=True)
    if param2 == "asc":
        sorted_df = df_spark.orderBy(parameter)
        break
    elif param2 == "desc":
        sorted_df = df_spark.orderBy(desc(parameter))
        break
    else:
        print("Error, please enter desc or asc")

#indicating which parameter that has been used to clarify the graph
print(f"Data sorted by using {parameter} as the reference")
sorted_df.show(howMany_shown)

Data sorted by using CS as the reference
+-------+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+
|   name|lastname|               email|      phone|             address|balance|credit_nb|     MS| CS|                uuid|
+-------+--------+--------------------+-----------+--------------------+-------+---------+-------+---+--------------------+
|  Bruce|   Wayne|notbatman@gotham.com|        911|1007 Mountain Dri...|  9.0E9|     2581| single|850|d410999a-0a62-400...|
| Nathan|  Player|  Fallout4@gmail.com|555-233-431|2, Sanctuary Hill...|    0.0|        0|widowed|850|6998d527-b849-484...|
|Charlie| Johnson|charlie.j@example...|555-234-569|789 Pine St, Spri...| 5000.0|        1| single|800|66f666a0-5326-424...|
| Victor|  Nickel|victor.nickel@exa...|555-234-588|123 Maple Circle,...| 8000.0|        1| single|800|03055554-6745-458...|
|   Yara|Aluminum|yara.aluminum@exa...|555-234-591|135 Cedar Circle,...| 6500.0|        1| 

## Create a folder and Store the Data into it :

In [ ]:
#Creating the file and naming it
fldr_name = input("Please enter a name for the Folder")
sorted_df.coalesce(1).write.csv(fldr_name, header=False, mode='overwrite')


## Stopping Borz0 :

In [27]:
#Stopping the ETL Pipeline to release memory and ressources
spark.stop()
#Checking if Spark is stoppped
try:
    if spark.sparkContext._jsc is None:
        print("Spark successfully stopped")
    else:
        print("Error: Spark is not stopped")
#if the upper part did not work :
except AttributeError:
    print("Spark successfully stopped")

Spark successfully stopped
